In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('drug_review_train.csv', engine='python', on_bad_lines='skip')
df.shape

(110811, 9)

In [ ]:
df

,Unnamed: 0,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,0,89879,Cyclosporine,keratoconjunctivitis sicca,"""i have used restasis for about a year now and...",2.0,"April 20, 2013",69,147
1,1,143975,Etonogestrel,birth control,"""my experience has been somewhat mixed. i have...",7.0,"August 7, 2016",4,136
2,2,106473,Implanon,birth control,"""this is my second implanon would not recommen...",1.0,"May 11, 2016",6,140
3,3,184526,Hydroxyzine,anxiety,"""i recommend taking as prescribed, and the bot...",10.0,"March 19, 2012",124,104
4,4,91587,Dalfampridine,multiple sclerosis,"""i have been on ampyra for 5 days and have bee...",9.0,"August 1, 2010",101,74
...,...,...,...,...,...,...,...,...,...
110806,806,140111,Asenapine,bipolar disorde,"""i had tried many medications to level out my ...",8.0,"September 14, 2016",18,141
110807,807,103534,Synthroid,underactive thyroid,"""had partial thyroid removal with benign tumor...",7.0,"October 27, 2014",61,124
110808,808,161014,Risperidone,depression,"""my experience with risperdal started when i f...",8.0,"September 2, 2015",21,130
110809,809,132424,Ativan,anxiety,"""my elderly dad was prescribed a low dose of a...",1.0,"March 30, 2009",47,77


# Data Preprocessing

Xóa cột 'Unnamed: 0'

In [ ]:
df= df.drop(columns=['Unnamed: 0'])

Xóa duplicates

In [ ]:
df.drop_duplicates(inplace=True)

Xóa URL & Email

In [ ]:
import re

In [ ]:
pd.set_option('display.max_colwidth', None)

# Hàm kiểm tra URL và email
def has_url_or_email(text):
    url_pattern = r'https?://\S+|www\.\S+'
    email_pattern = r'\S+@\S+'

    return bool(re.search(url_pattern, text)) or bool(re.search(email_pattern, text))

# Lọc ra các dòng KHÔNG chứa URL hoặc email
df = df[~df['review'].apply(has_url_or_email)].copy()

# In số lượng review còn lại
print(f"Số lượng review sau khi xóa các dòng có URL hoặc Email: {df.shape[0]}")

Số lượng review sau khi xóa các dòng có URL hoặc Email: 110776


In [ ]:
df.shape

(110776, 9)

HTML

In [ ]:
html_entities_pattern = r'&[a-zA-Z]+;'

has_html_entities = df['review'].str.contains(html_entities_pattern).any()

print("Có HTML entities trong review không?", has_html_entities)

Có HTML entities trong review không? False


In [ ]:
def contains_non_ascii(text):
    return bool(re.search(r'[^\x00-\x7F]+', text))

# Kiểm tra tất cả các review trong cột 'review' có ký tự phi ASCII
df['contains_non_ascii'] = df['review'].apply(contains_non_ascii)

# Lọc và xóa các dòng có chứa ký tự phi ASCII
df = df[~df['contains_non_ascii']].copy()

# In số lượng các review còn lại
print(f"Số lượng review sau khi xóa các dòng có ký tự phi ASCII: {df.shape[0]}")

Số lượng review sau khi xóa các dòng có ký tự phi ASCII: 109682


### Tokenisation

In [ ]:
!pip install nltk


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize

df['tokens'] = df['review'].apply(lambda x: word_tokenize(str(x)) if pd.notnull(x) else [])


In [ ]:
df.head(3)

,Unnamed: 0,patient_id,drugName,condition,review,rating,date,usefulCount,review_length,contains_non_ascii,tokens
0,0,89879,Cyclosporine,keratoconjunctivitis sicca,"""i have used restasis for about a year now and have seen almost no progress. for most of my life i've had red and bothersome eyes. after trying various eye drops, my doctor recommended restasis. he said it typically takes 3 to 6 months for it to really kick in but it never did kick in. when i put the drops in it burns my eyes for the first 30 - 40 minutes. i've talked with my doctor about this and he said it is normal but should go away after some time, but it hasn't. every year around spring time my eyes get terrible irritated and this year has been the same (maybe even worse than other years) even though i've been using restasis for a year now. the only difference i notice was for the first couple weeks, but now i'm ready to move on.""",2.0,"April 20, 2013",69,147,False,"[``, i, have, used, restasis, for, about, a, year, now, and, have, seen, almost, no, progress, ., for, most, of, my, life, i, 've, had, red, and, bothersome, eyes, ., after, trying, various, eye, drops, ,, my, doctor, recommended, restasis, ., he, said, it, typically, takes, 3, to, 6, months, for, it, to, really, kick, in, but, it, never, did, kick, in, ., when, i, put, the, drops, in, it, burns, my, eyes, for, the, first, 30, -, 40, minutes, ., i, 've, talked, with, my, doctor, about, this, and, he, said, it, is, normal, but, should, go, away, after, ...]"
1,1,143975,Etonogestrel,birth control,"""my experience has been somewhat mixed. i have been using implanon now for nearly 14 months and have decided to get it removed because i bleed every day, all day. i would occasionally stain my underwear and my sheets. it didn't start out that way, for the first month i didn't bleed at all, then i had an epic two week period, then everything was irregular, which has basically been my new norm, sadly. \r\r\n\r\r\ni decided to get rid of the implanon because of the endless bleeding. i should mention that the bleeding was usually pretty light, a bit more than spotting but not quite a period either. other than the endless bleeding, i've been pretty side effect free, except for minor acne.\r\r\n\r\r\ni didn't get pregnant on it, so yeah! but...blood, a lot of blood.""",7.0,"August 7, 2016",4,136,False,"[``, my, experience, has, been, somewhat, mixed, ., i, have, been, using, implanon, now, for, nearly, 14, months, and, have, decided, to, get, it, removed, because, i, bleed, every, day, ,, all, day, ., i, would, occasionally, stain, my, underwear, and, my, sheets, ., it, did, n't, start, out, that, way, ,, for, the, first, month, i, did, n't, bleed, at, all, ,, then, i, had, an, epic, two, week, period, ,, then, everything, was, irregular, ,, which, has, basically, been, my, new, norm, ,, sadly, ., i, decided, to, get, rid, of, the, implanon, because, of, the, endless, bleeding, ...]"
2,2,106473,Implanon,birth control,"""this is my second implanon would not recommend at all....first one was okay for the first 2 years until i started bleeding which never stopped when i woke in the morning i would stand up and be absolutely covered in blood it was that bad.....i lost 2 stone in weight my mood swings were all over the place and it affected my relationship. i was advised to have that one removed and another one inserted which i did which i'm into the second year now and it's starting all over again! the only good thing about this is the reliability that your not going to get pregnant if there's ever a chance that you can have a quick fumble in between periods!! it gets me so down i can be off in the morning and on again at night.....so frustrating""",1.0,"May 11, 2016",6,140,False,"[``, this, is, my, second, implanon, would, not, recommend, at, all, ...., first, one, was, okay, for, the, first, 2, years, until, i, started, bleeding, which, never, stopped, when, i, woke, in, the, morning, i, would, stand, up, and, b

### Xử lý sơ bộ

In [ ]:
import re

def token_clean_full(tokens):
    cleaned_tokens = []

    for token in tokens:
        # Lowercase
        token = token.lower()

        # Remove non-ASCII characters
        token = re.sub(r'[^\x00-\x7F]+', ' ', token)

        # Remove special characters (keep only letters, numbers, and whitespace)
        token = re.sub(r'[^\w\s\']', ' ', token)

        # Remove multiple dots and individual dot tokens
        token = re.sub(r'\.{2,}', ' ', token)
        if token.strip() == ".":
            continue

        # Strip leading/trailing spaces
        token = token.strip()

        # Remove empty tokens and pure whitespace
        if token and not token.isspace():
            cleaned_tokens.append(token)

    return cleaned_tokens

# Apply to DataFrame
df['token_clean'] = df['tokens'].apply(token_clean_full)


In [ ]:
df['token_clean'].head(10)

,token_clean
0,"[i, have, used, restasis, for, about, a, year, now, and, have, seen, almost, no, progress, for, most, of, my, life, i, 've, had, red, and, bothersome, eyes, after, trying, various, eye, drops, my, doctor, recommended, restasis, he, said, it, typically, takes, 3, to, 6, months, for, it, to, really, kick, in, but, it, never, did, kick, in, when, i, put, the, drops, in, it, burns, my, eyes, for, the, first, 30, 40, minutes, i, 've, talked, with, my, doctor, about, this, and, he, said, it, is, normal, but, should, go, away, after, some, time, but, it, has, n't, every, year, ...]"
1,"[my, experience, has, been, somewhat, mixed, i, have, been, using, implanon, now, for, nearly, 14, months, and, have, decided, to, get, it, removed, because, i, bleed, every, day, all, day, i, would, occasionally, stain, my, underwear, and, my, sheets, it, did, n't, start, out, that, way, for, the, first, month, i, did, n't, bleed, at, all, then, i, had, an, epic, two, week, period, then, everything, was, irregular, which, has, basically, been, my, new, norm, sadly, i, decided, to, get, rid, of, the, implanon, because, of, the, endless, bleeding, i, should, mention, that, the, bleeding, was, usually, pretty, light, a, ...]"
2,"[this, is, my, second, implanon, would, not, recommend, at, all, first, one, was, okay, for, the, first, 2, years, until, i, started, bleeding, which, never, stopped, when, i, woke, in, the, morning, i, would, stand, up, and, be, absolutely, covered, in, blood, it, was, that, bad, i, lost, 2, stone, in, weight, my, mood, swings, were, all, over, the, place, and, it, affected, my, relationship, i, was, advised, to, have, that, one, removed, and, another, one, inserted, which, i, did, which, i, 'm, into, the, second, year, now, and, it, 's, starting, all, over, again, the, only, good, thing, about, ...]"
3,"[i, recommend, taking, as, prescribed, and, the, bottle, usually, says, take, x, amount, every, x, hours, '', i, think, that, having, a, steady, stream, of, any, medication, is, the, only, way, to, have, it, work, this, medication, 's, only, side, effect, i, have, found, is, almost, exactly, like, when, one, takes, benadryl, and, only, when, you, are, first, starting, out, on, it, i, am, used, to, it, now, after, 3, weeks, and, the, only, effect, i, notice, is, that, my, hands, no, longer, shake, and, my, mind, is, at, ease, much, better, and, smoother, than, other, meds, like, xanax, or, ...]"
4,"[i, have, been, on, ampyra, for, 5, days, and, have, been, so, happy, with, this, new, pill, the, first, 2, days, were, not, good, with, the, side, effects, but, each, day, just, gets, better, i, am, able, to, walk, better, without, dragging, my, left, foot, and, get, up, from, a, kneeling, position, without, to, much, effort, this, new, medicine, has, given, me, a, new, outlook, on, my, future, thank, you, for, this, medical, wonder, '']"
5,"[used, for, birth, control, and, period, issues, very, unhappy, worked, good, for, preventing, pregnancy, along, with, terrible, side, effects, hard, time, sleeping, if, any, nightmares, anxiety, depression, the, list, goes, on, everybody, is, different, i, recommend, trying, this, pill, to, see, if, it, 's, for, you, it, just, was, n't, for, me, switching, back, to, my, old, birth, control, '']"
6,"[my, prep, instructions, were, one, 6oz, bottle, the, evening, before, and, one, 6oz, bottle, the, morning, of, the, colonoscopy, both, times, were, similar, started, working, within, 20 25, minutes, and, lasted, a, couple, hours, i, mixed, with, sprite, the, review, that, mentions, salty, grape, cough, syrup, was, spot, on, the, first, bottle, was, n't, terrible, the, second, one, almost, made, me, puke, compared, to, other, prep, experiences, i, 've, heard, outside, of, this, site, i, think, suprep, is, probably, one, of, the, easier, ones, of, course, everyone, reacts, different, '']"
7,"[love, it, i, had, the, worst, breakouts, so, i, got, epiduo, when, i, used, it, the, first, time

### Xử lý ngôn ngữ

Contraction

In [ ]:
import re

contraction_dict = {
    "n't": "not","can't": "cannot", "'cause": "because",
    "'ve": "have", "'d": "would",
    "'ll": "will", "'s": "is", "'m": "am","'s": "us",
    "'re": "are","wo": "will"
}

# Compile regex for contractions
contractions_re = re.compile('(%s)' % '|'.join(map(re.escape, contraction_dict.keys())))

# Hàm thay thế contraction cho list token
def replace_contractions_tokens(tokens):
    return [contraction_dict[token] if token in contraction_dict else token for token in tokens]

# Apply to 'token_clean' column
df['token_clean_contracted'] = df['token_clean'].apply(replace_contractions_tokens)
df['token_clean'] = df['token_clean_contracted']


In [ ]:
df['token_clean'] = df['token_clean_contracted']
df['token_clean'].head(5)

,token_clean
0,"[i, have, used, restasis, for, about, a, year, now, and, have, seen, almost, no, progress, for, most, of, my, life, i, have, had, red, and, bothersome, eyes, after, trying, various, eye, drops, my, doctor, recommended, restasis, he, said, it, typically, takes, 3, to, 6, months, for, it, to, really, kick, in, but, it, never, did, kick, in, when, i, put, the, drops, in, it, burns, my, eyes, for, the, first, 30, 40, minutes, i, have, talked, with, my, doctor, about, this, and, he, said, it, is, normal, but, should, go, away, after, some, time, but, it, has, not, every, year, ...]"
1,"[my, experience, has, been, somewhat, mixed, i, have, been, using, implanon, now, for, nearly, 14, months, and, have, decided, to, get, it, removed, because, i, bleed, every, day, all, day, i, would, occasionally, stain, my, underwear, and, my, sheets, it, did, not, start, out, that, way, for, the, first, month, i, did, not, bleed, at, all, then, i, had, an, epic, two, week, period, then, everything, was, irregular, which, has, basically, been, my, new, norm, sadly, i, decided, to, get, rid, of, the, implanon, because, of, the, endless, bleeding, i, should, mention, that, the, bleeding, was, usually, pretty, light, a, ...]"
2,"[this, is, my, second, implanon, would, not, recommend, at, all, first, one, was, okay, for, the, first, 2, years, until, i, started, bleeding, which, never, stopped, when, i, woke, in, the, morning, i, would, stand, up, and, be, absolutely, covered, in, blood, it, was, that, bad, i, lost, 2, stone, in, weight, my, mood, swings, were, all, over, the, place, and, it, affected, my, relationship, i, was, advised, to, have, that, one, removed, and, another, one, inserted, which, i, did, which, i, am, into, the, second, year, now, and, it, us, starting, all, over, again, the, only, good, thing, about, ...]"
3,"[i, recommend, taking, as, prescribed, and, the, bottle, usually, says, take, x, amount, every, x, hours, '', i, think, that, having, a, steady, stream, of, any, medication, is, the, only, way, to, have, it, work, this, medication, us, only, side, effect, i, have, found, is, almost, exactly, like, when, one, takes, benadryl, and, only, when, you, are, first, starting, out, on, it, i, am, used, to, it, now, after, 3, weeks, and, the, only, effect, i, notice, is, that, my, hands, no, longer, shake, and, my, mind, is, at, ease, much, better, and, smoother, than, other, meds, like, xanax, or, ...]"
4,"[i, have, been, on, ampyra, for, 5, days, and, have, been, so, happy, with, this, new, pill, the, first, 2, days, were, not, good, with, the, side, effects, but, each, day, just, gets, better, i, am, able, to, walk, better, without, dragging, my, left, foot, and, get, up, from, a, kneeling, position, without, to, much, effort, this, new, medicine, has, given, me, a, new, outlook, on, my, future, thank, you, for, this, medical, wonder, '']"


stop word

In [ ]:
pip install nltk


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Lấy danh sách stopwords
stop_words = set(stopwords.words('english'))

# Hàm loại bỏ stopwords cho cột token_clean
def remove_stopwords_tokens(tokens):
    # Lọc bỏ các stopwords
    filtered_tokens = [token for token in tokens if token not in stop_words]

    return filtered_tokens

# Áp dụng hàm cho cột token_clean
df['token_clean_no_stopwords'] = df['token_clean'].apply(remove_stopwords_tokens)

# Nếu cần, bạn có thể thay thế cột token_clean bằng token_clean_no_stopwords
df['token_clean'] = df['token_clean_no_stopwords']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df['token_clean'].head(5)

,token_clean
0,"[used, restasis, year, seen, almost, progress, life, red, bothersome, eyes, trying, various, eye, drops, doctor, recommended, restasis, said, typically, takes, 3, 6, months, really, kick, never, kick, put, drops, burns, eyes, first, 30, 40, minutes, talked, doctor, said, normal, go, away, time, every, year, around, spring, time, eyes, get, terrible, irritated, year, maybe, even, worse, years, even, though, using, restasis, year, difference, notice, first, couple, weeks, ready, move, '']"
1,"[experience, somewhat, mixed, using, implanon, nearly, 14, months, decided, get, removed, bleed, every, day, day, would, occasionally, stain, underwear, sheets, start, way, first, month, bleed, epic, two, week, period, everything, irregular, basically, new, norm, sadly, decided, get, rid, implanon, endless, bleeding, mention, bleeding, usually, pretty, light, bit, spotting, quite, period, either, endless, bleeding, pretty, side, effect, free, except, minor, acne, get, pregnant, yeah, blood, lot, blood, '']"
2,"[second, implanon, would, recommend, first, one, okay, first, 2, years, started, bleeding, never, stopped, woke, morning, would, stand, absolutely, covered, blood, bad, lost, 2, stone, weight, mood, swings, place, affected, relationship, advised, one, removed, another, one, inserted, second, year, us, starting, good, thing, reliability, going, get, pregnant, us, ever, chance, quick, fumble, periods, gets, morning, night, frustrating, '']"
3,"[recommend, taking, prescribed, bottle, usually, says, take, x, amount, every, x, hours, '', think, steady, stream, medication, way, work, medication, us, side, effect, found, almost, exactly, like, one, takes, benadryl, first, starting, used, 3, weeks, effect, notice, hands, longer, shake, mind, ease, much, better, smoother, meds, like, xanax, tramadol, klonopin, etc, etc]"
4,"[ampyra, 5, days, happy, new, pill, first, 2, days, good, side, effects, day, gets, better, able, walk, better, without, dragging, left, foot, get, kneeling, position, without, much, effort, new, medicine, given, new, outlook, future, thank, medical, wonder, '']"


POS tagging

In [ ]:
import nltk
from nltk import pos_tag

# Tải tài nguyên cần thiết
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')  # Tải tài nguyên còn thiếu

# Giả sử df['token_clean'] chứa danh sách các từ (tokens)
# Áp dụng POS tagging cho cột token_clean
df['pos_tagged'] = df['token_clean'].apply(pos_tag)

# In kết quả
print(df['pos_tagged'].head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


0    [(used, VBN), (restasis, NN), (year, NN), (seen, VBN), (almost, RB), (progress, JJ), (life, NN), (red, VBD), (bothersome, JJ), (eyes, NNS), (trying, VBG), (various, JJ), (eye, NN), (drops, NNS), (doctor, NN), (recommended, VBD), (restasis, NN), (said, VBD), (typically, RB), (takes, VBZ), (3, CD), (6, CD), (months, NNS), (really, RB), (kick, VBP), (never, RB), (kick, VBN), (put, NN), (drops, NNS), (burns, NNS), (eyes, NNS), (first, RB), (30, CD), (40, CD), (minutes, NNS), (talked, VBD), (doctor, NN), (said, VBD), (normal, JJ), (go, VB), (away, RB), (time, NN), (every, DT), (year, NN), (around, IN), (spring, NN), (time, NN), (eyes, NNS), (get, VBP), (terrible, JJ), (irritated, JJ), (year, NN), (maybe, RB), (even, RB), (worse, JJR), (years, NNS), (even, RB), (though, IN), (using, VBG), (restasis, NN), (year, NN), (difference, NN), (notice, NN), (first, RB), (couple, JJ), (weeks, NNS), (ready, JJ), (move, NN), ('', '')]
1          [(experience, NN), (somewhat, RB), (mixed, JJ), (using

Lemmatization

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 104.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Tải các tài nguyên cần thiết nếu chưa có
nltk.download('wordnet')
nltk.download('omw-1.4')  # Đảm bảo tải tài nguyên của WordNet

# Khởi tạo lemmatizer
lemmatizer = WordNetLemmatizer()

# Hàm chuyển POS tag từ NLTK sang WordNet
def get_wordnet_pos(nltk_pos):
    if nltk_pos.startswith('V'):
        return wordnet.VERB
    elif nltk_pos.startswith('N'):
        return wordnet.NOUN
    elif nltk_pos.startswith('J'):
        return wordnet.ADJ
    elif nltk_pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Mặc định là danh từ nếu không xác định được

# Hàm lemmatize cho từng token với POS tag
def lemmatize_tokens_with_pos(tokens_pos):
    lemmatized_tokens = []
    for token, pos in tokens_pos:
        wordnet_pos = get_wordnet_pos(pos)
        lemmatized_token = lemmatizer.lemmatize(token, wordnet_pos)
        lemmatized_tokens.append(lemmatized_token)
    return lemmatized_tokens

# Áp dụng lemmatization cho cột pos_tagged
df['lemmatized'] = df['pos_tagged'].apply(lemmatize_tokens_with_pos)

# In kết quả
print(df[['pos_tagged', 'lemmatized']].head())


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          pos_tagged  \
0  [(used, VBN), (restasis, NN), (year, NN), (seen, VBN), (almos

In [ ]:
df.head()

,Unnamed: 0,patient_id,drugName,condition,review,rating,date,usefulCount,review_length,contains_non_ascii,tokens,token_clean,token_clean_contracted,token_clean_no_stopwords,pos_tagged,lemmatized
0,0,89879,Cyclosporine,keratoconjunctivitis sicca,"""i have used restasis for about a year now and have seen almost no progress. for most of my life i've had red and bothersome eyes. after trying various eye drops, my doctor recommended restasis. he said it typically takes 3 to 6 months for it to really kick in but it never did kick in. when i put the drops in it burns my eyes for the first 30 - 40 minutes. i've talked with my doctor about this and he said it is normal but should go away after some time, but it hasn't. every year around spring time my eyes get terrible irritated and this year has been the same (maybe even worse than other years) even though i've been using restasis for a year now. the only difference i notice was for the first couple weeks, but now i'm ready to move on.""",2.0,"April 20, 2013",69,147,False,"[``, i, have, used, restasis, for, about, a, year, now, and, have, seen, almost, no, progress, ., for, most, of, my, life, i, 've, had, red, and, bothersome, eyes, ., after, trying, various, eye, drops, ,, my, doctor, recommended, restasis, ., he, said, it, typically, takes, 3, to, 6, months, for, it, to, really, kick, in, but, it, never, did, kick, in, ., when, i, put, the, drops, in, it, burns, my, eyes, for, the, first, 30, -, 40, minutes, ., i, 've, talked, with, my, doctor, about, this, and, he, said, it, is, normal, but, should, go, away, after, ...]","[used, restasis, year, seen, almost, progress, life, red, bothersome, eyes, trying, various, eye, drops, doctor, recommended, restasis, said, typically, takes, 3, 6, months, really, kick, never, kick, put, drops, burns, eyes, first, 30, 40, minutes, talked, doctor, said, normal, go, away, time, every, year, around, spring, time, eyes, get, terrible, irritated, year, maybe, even, worse, years, even, though, using, restasis, year, difference, notice, first, couple, weeks, ready, move, '']","[i, have, used, restasis, for, about, a, year, now, and, have, seen, almost, no, progress, for, most, of, my, life, i, have, had, red, and, bothersome, eyes, after, trying, various, eye, drops, my, doctor, recommended, restasis, he, said, it, typically, takes, 3, to, 6, months, for, it, to, really, kick, in, but, it, never, did, kick, in, when, i, put, the, drops, in, it, burns, my, eyes, for, the, first, 30, 40, minutes, i, have, talked, with, my, doctor, about, this, and, he, said, it, is, normal, but, should, go, away, after, some, time, but, it, has, not, every, year, ...]","[used, restasis, year, seen, almost, progress, life, red, bothersome, eyes, trying, various, eye, drops, doctor, recommended, restasis, said, typically, takes, 3, 6, months, really, kick, never, kick, put, drops, burns, eyes, first, 30, 40, minutes, talked, doctor, said, normal, go, away, time, every, year, around, spring, time, eyes, get, terrible, irritated, year, maybe, even, worse, years, even, though, using, restasis, year, difference, notice, first, couple, weeks, ready, move, '']","[(used, VBN), (restasis, NN), (year, NN), (seen, VBN), (almost, RB), (progress, JJ), (life, NN), (red, VBD), (bothersome, JJ), (eyes, NNS), (trying, VBG), (various, JJ), (eye, NN), (drops, NNS), (doctor, NN), (recommended, VBD), (restasis, NN), (said, VBD), (typically, RB), (takes, VBZ), (3, CD), (6, CD), (months, NNS), (really, RB), (kick, VBP), (never, RB), (kick, VBN), (put, NN), (drops, NNS), (burns, NNS), (eyes, NNS), (first, RB), (30, CD), (40, CD), (minutes, NNS), (talked, VBD), (doctor, NN), (said, VBD), (normal, JJ), (go, VB), (away, RB), (time, NN), (every, DT), (year, NN), (around, IN), (spring, NN), (time, NN), (eyes, NNS), (get, VBP), (terrible, JJ), (irritated, JJ), (year, NN), (maybe, RB), (even, RB), (worse, JJR), (years, NNS), (even, RB), (though, IN), (using, VBG), (restasis, NN), (year, NN), (differe

In [ ]:
!pip install textblob
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:
from collections import Counter

# Tạo danh sách tất cả các từ từ cột 'lemmatized'
all_tokens = [token for tokens in df['lemmatized'] for token in tokens]

# Tính tần suất xuất hiện
word_freq = Counter(all_tokens)

# Lọc các từ có tần suất < 3
rare_words = [word for word, freq in word_freq.items() if freq < 3]

# In danh sách từ hiếm và tổng số lượng
print("Các từ hiếm:", rare_words)
print(f"Tổng số từ hiếm: {len(rare_words)}")

Các từ hiếm: ['merge', 'shot my', '1989 90', 'buoyant', 'ever diminishing', 'parenting', 'inadvertently', 'weepies', 'trapezoid', '2inx', '1inch', 'force do', '3 in 1', 'anxiety agitation', '1 23', '05 15 12', 'pain gas', 'aced', 'propranonol', 'itichy', 'dectable', 'diverse', 'spiked', 'nylan', 'mallinckrodt covidien', 'mirvasa', 'chemo radiation', 'torn ruptured', 'skinner', 'pysc', 'nevr', 'itraconozol', 'dremel', 'festerd', 'blatant', 'moodswing', 'detected', 'pcod', 'meformin', 'almased', 'maitake', 'apron', '3x300mg', '600x2', '300x1', 'xologel', 'ketaconazole', 'thrombosed', 'antibx', 'cefdiner', 'june july', '2 x', 'b cup', 'hrs  no', 'gum nerve', 'fearsome', 'wounder', 'prolea', 'vascular aura', 'w tests', 'anolor', "'preventative", 'life job', 'ui', 'yasmin which', 'ethereal', '92 00', 'adepex', '400 500mg', 'yanked', 'inteaction', 'fobromyalgia', 'definatley', 'effective painless', 'rejoicing', 'nastiest', 'mattify', 'disruption insomnia', 'oftentimes', 'angry depressed', 'd

In [ ]:
print("Danh sách từ hiếm kèm tần suất:")
for word in rare_words:
    print(f"{word}: {word_freq[word]}")

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
terrifically: 1
disquiet: 1
cholestral: 1
krill: 2
unwillingness: 1
walzyr: 1
citirizine: 1
metrorrage: 1
27 29: 1
35 50: 1
pimpled: 2
streph: 1
compartmentalize: 2
construct: 2
erin: 1
wakefuleness: 1
straight i: 1
lgot: 1
rinds: 1
rotisserie: 1
inhere: 1
down was: 1
'push: 1
stings: 1
uti i: 1
bci: 1
cycle spotting: 1
pressure stroke: 1
exspect: 1
sitrate: 1
relef: 1
felodopine: 1
parmidxl: 1
hrs week: 1
1200 1500 day: 1
diction: 1
ups by: 1
8mg 2: 1
undetecable: 2
v load: 2
725: 2
andrgel: 1
hardwired: 1
endeavour: 1
iroman: 1
licrogel: 2
800mh: 1
122kg: 1
say you: 1
use abuse: 1
somemore: 2
hooooray: 1
4 28 16: 1
lonox: 1
frenzied: 1
enemas suppositories: 1
12 04 14: 1
259lbs: 1
pressure high: 1
finaily: 2
highways: 1
sertealine: 2
excitalopram: 1
fully now: 1
116 126 70: 1
142 155 90: 1
q 10: 1
phytosterol: 1
areds2: 2
non ambulatory: 1
6 8 16: 2
siked: 1
author: 1
ulcerartive: 1
lowers: 1
zytiga: 2
inadvertent: 1
meta analy

In [ ]:
from textblob import TextBlob

# Hàm sửa lỗi chính tả nếu có
def correct_if_misspelled(word):
    corrected = str(TextBlob(word).correct())
    return corrected if corrected != word else None  # Chỉ trả về nếu bị sửa

# Tìm từ hiếm có khả năng sai chính tả
misspelled_corrections = {
    word: correct_if_misspelled(word)
    for word in rare_words
    if correct_if_misspelled(word)  # chỉ lấy những từ bị sửa
}

# In kết quả
print("Các từ sai chính tả (từ hiếm):")
for wrong, corrected in misspelled_corrections.items():
    print(f"{wrong} -> {corrected}")

print(f"\nTổng số từ sai chính tả được phát hiện: {len(misspelled_corrections)}")

Các từ sai chính tả (từ hiếm):
parenting -> printing
weepies -> weeping
2inx -> in
1inch -> inch
aced -> acid
itichy -> itchy
spiked -> spike
nylan -> plan
chemo radiation -> them radiation
skinner -> sinner
pysc -> past
nevr -> never
dremel -> dreamed
festerd -> fester
blatant -> latent
pcod -> cod
meformin -> reforming
almased -> pleased
maitake -> mistake
xologel -> colonel
antibx -> anti
hrs  no -> his  no
fearsome -> wearisome
wounder -> wounded
prolea -> prove
vascular aura -> vascular laura
anolor -> color
'preventative -> 'preventive
ui -> i
yasmin which -> basin which
ethereal -> eternal
adepex -> apex
yanked -> yankee
inteaction -> interaction
definatley -> definitely
nastiest -> easiest
mattify -> ratify
drys -> days
recooperative -> cooperative
20mph -> lymph
cuboid -> mucoid
m37 -> may
3 5lbs -> 3 sobs
wonderfull -> wonderful
asb -> as
besfor -> before
mivaso -> vaso
waa -> was
mire -> more
gor -> for
excercise eat -> exercise eat
combo mono -> comb mon
tranquilliser -> tr

## Save file


In [ ]:
# Lưu file
df.to_csv("clean_data.csv", index=False)

# Tải file
from google.colab import files
files.download("clean_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df.head()

,Unnamed: 0,patient_id,drugName,condition,review,rating,date,usefulCount,review_length,contains_non_ascii,tokens,token_clean,token_clean_contracted,token_clean_no_stopwords,pos_tagged,lemmatized
0,0,89879,Cyclosporine,keratoconjunctivitis sicca,"""i have used restasis for about a year now and have seen almost no progress. for most of my life i've had red and bothersome eyes. after trying various eye drops, my doctor recommended restasis. he said it typically takes 3 to 6 months for it to really kick in but it never did kick in. when i put the drops in it burns my eyes for the first 30 - 40 minutes. i've talked with my doctor about this and he said it is normal but should go away after some time, but it hasn't. every year around spring time my eyes get terrible irritated and this year has been the same (maybe even worse than other years) even though i've been using restasis for a year now. the only difference i notice was for the first couple weeks, but now i'm ready to move on.""",2.0,"April 20, 2013",69,147,False,"[``, i, have, used, restasis, for, about, a, year, now, and, have, seen, almost, no, progress, ., for, most, of, my, life, i, 've, had, red, and, bothersome, eyes, ., after, trying, various, eye, drops, ,, my, doctor, recommended, restasis, ., he, said, it, typically, takes, 3, to, 6, months, for, it, to, really, kick, in, but, it, never, did, kick, in, ., when, i, put, the, drops, in, it, burns, my, eyes, for, the, first, 30, -, 40, minutes, ., i, 've, talked, with, my, doctor, about, this, and, he, said, it, is, normal, but, should, go, away, after, ...]","[used, restasis, year, seen, almost, progress, life, red, bothersome, eyes, trying, various, eye, drops, doctor, recommended, restasis, said, typically, takes, 3, 6, months, really, kick, never, kick, put, drops, burns, eyes, first, 30, 40, minutes, talked, doctor, said, normal, go, away, time, every, year, around, spring, time, eyes, get, terrible, irritated, year, maybe, even, worse, years, even, though, using, restasis, year, difference, notice, first, couple, weeks, ready, move, '']","[i, have, used, restasis, for, about, a, year, now, and, have, seen, almost, no, progress, for, most, of, my, life, i, have, had, red, and, bothersome, eyes, after, trying, various, eye, drops, my, doctor, recommended, restasis, he, said, it, typically, takes, 3, to, 6, months, for, it, to, really, kick, in, but, it, never, did, kick, in, when, i, put, the, drops, in, it, burns, my, eyes, for, the, first, 30, 40, minutes, i, have, talked, with, my, doctor, about, this, and, he, said, it, is, normal, but, should, go, away, after, some, time, but, it, has, not, every, year, ...]","[used, restasis, year, seen, almost, progress, life, red, bothersome, eyes, trying, various, eye, drops, doctor, recommended, restasis, said, typically, takes, 3, 6, months, really, kick, never, kick, put, drops, burns, eyes, first, 30, 40, minutes, talked, doctor, said, normal, go, away, time, every, year, around, spring, time, eyes, get, terrible, irritated, year, maybe, even, worse, years, even, though, using, restasis, year, difference, notice, first, couple, weeks, ready, move, '']","[(used, VBN), (restasis, NN), (year, NN), (seen, VBN), (almost, RB), (progress, JJ), (life, NN), (red, VBD), (bothersome, JJ), (eyes, NNS), (trying, VBG), (various, JJ), (eye, NN), (drops, NNS), (doctor, NN), (recommended, VBD), (restasis, NN), (said, VBD), (typically, RB), (takes, VBZ), (3, CD), (6, CD), (months, NNS), (really, RB), (kick, VBP), (never, RB), (kick, VBN), (put, NN), (drops, NNS), (burns, NNS), (eyes, NNS), (first, RB), (30, CD), (40, CD), (minutes, NNS), (talked, VBD), (doctor, NN), (said, VBD), (normal, JJ), (go, VB), (away, RB), (time, NN), (every, DT), (year, NN), (around, IN), (spring, NN), (time, NN), (eyes, NNS), (get, VBP), (terrible, JJ), (irritated, JJ), (year, NN), (maybe, RB), (even, RB), (worse, JJR), (years, NNS), (even, RB), (though, IN), (using, VBG), (restasis, NN), (year, NN), (differe